**保存和加载模型**

当保存和加载模型时，需要熟悉三个核心功能：

1.torch.save：将序列化对象保存到磁盘。此函数使用Python的pickle模块进行序列化。使用此函数可以保存如模型、tensor、字典等各种对象。

2.torch.load：使用pickle的unpickling功能将pickle对象文件反序列化到内存。此功能还可以有助于设备加载数据。

3.torch.nn.Module.load_state_dict：使用反序列化函数 state_dict 来加载模型的参数字典。

In [ ]:
#1.什么是状态字典：state_dict?
# 定义模型
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 初始化模型
model = TheModelClass()

# 初始化优化器
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 打印模型的状态字典
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# 打印优化器的状态字典
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
#2.1 保存/加载state_dict（推荐使用）
torch.save(model.state_dict(), PATH)

#加载
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

In [ ]:
#2.2 保存/加载完整模型
torch.save(model, PATH)

#加载
# 模型类必须在此之前被定义
model = torch.load(PATH)
model.eval()

In [ ]:
#3. 保存和加载 Checkpoint 用于推理/继续训练
#保存
torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss': loss,
      ...
      }, PATH)

#加载
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()

In [ ]:
#4. 在一个文件中保存多个模型
#保存
torch.save({
      'modelA_state_dict': modelA.state_dict(),
      'modelB_state_dict': modelB.state_dict(),
      'optimizerA_state_dict': optimizerA.state_dict(),
      'optimizerB_state_dict': optimizerB.state_dict(),
      ...
      }, PATH)

#加载
modelA = TheModelAClass(*args, **kwargs)
modelB = TheModelBClass(*args, **kwargs)
optimizerA = TheOptimizerAClass(*args, **kwargs)
optimizerB = TheOptimizerBClass(*args, **kwargs)

checkpoint = torch.load(PATH)
modelA.load_state_dict(checkpoint['modelA_state_dict'])
modelB.load_state_dict(checkpoint['modelB_state_dict'])
optimizerA.load_state_dict(checkpoint['optimizerA_state_dict'])
optimizerB.load_state_dict(checkpoint['optimizerB_state_dict'])

modelA.eval()
modelB.eval()
# - or -
modelA.train()
modelB.train()

In [ ]:
#5. 使用在不同模型参数下的热启动模式
#保存
torch.save(modelA.state_dict(), PATH)

#加载
modelB = TheModelBClass(*args, **kwargs)
modelB.load_state_dict(torch.load(PATH), strict=False)

In [ ]:
#6.1 保存到 CPU、加载到 CPU
#保存
torch.save(model.state_dict(), PATH)

#加载
device = torch.device('cpu')
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location=device))

In [ ]:
#6.2 保存到 GPU、加载到 GPU
#保存
torch.save(model.state_dict(), PATH)

#加载
device = torch.device("cuda")
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.to(device)
# 确保在你提供给模型的任何输入张量上调用input = input.to(device)

In [ ]:
#6.3 保存到 CPU，加载到 GPU
#保存
torch.save(model.state_dict(), PATH)

#加载
device = torch.device("cuda")
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location="cuda:0"))  # Choose whatever GPU device number you want
model.to(device)
# 确保在你提供给模型的任何输入张量上调用input = input.to(device)

In [ ]:
#6.4 保存 torch.nn.DataParallel 模型
#保存
torch.save(model.module.state_dict(), PATH)

#加载
#加载任何你想要的设备